# Laboratorium 5: Rekurencyjne Sieci Neuronowe (RNN)
## Część 2: roman2number

In [1]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector, Dropout
import numpy as np
from roman_numerals import convert

### Tworznenie modelu

In [2]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1),return_sequences=True)) # sequences of singlen numbers
model.add(LSTM(128))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 198,273
Trainable params: 198,273
Non-trainable params: 0
_________________________________________________________________


### Przygotowanie danych

In [3]:
DATASET_SIZE=200

samples = []
labels = []
all_words = ' ' # Puste pole dla liczb krótszych w zapisie
max_len = 0
for i in range(DATASET_SIZE):
    labels.append(i)
    words = convert(i)
    samples.append(words)
    all_words += words
    if len(words)>max_len: 
        max_len = len(words)
  
print('Max len of text',max_len)
vocab = sorted(set(all_words))
vocab_size = len(vocab)
print('vocabulary (used letters)',vocab)
print ('unique characters',vocab_size)

Max len of text 9
vocabulary (used letters) [' ', 'C', 'I', 'L', 'V', 'X']
unique characters 6


### Metody pomocnicze

In [4]:
# helper method, converts sequence of numbers to text
def to_text(sample):
    return ''.join([idx2char[int(x)] for x in sample])
# helper method, converts text to sequence of numbers
def to_number(words):
    return np.array([char2idx[char] for char in words])

### Mapowanie

In [5]:
char2idx = {char:index for index, char in enumerate(vocab)}
print('char2idx:\n',char2idx)
idx2char = np.array(vocab)
print('idx2char\n',idx2char)

char2idx:
 {' ': 0, 'C': 1, 'I': 2, 'L': 3, 'V': 4, 'X': 5}
idx2char
 [' ' 'C' 'I' 'L' 'V' 'X']


### Konwersja cyfr rzymskich na liczby

In [6]:
samples_int = []
for s in samples:
    v = np.array([char2idx[char] for char in s])
    samples_int.append(v) # different sizes!
print(samples[123],' ->becomes-> ',samples_int[123])

CXXIII  ->becomes->  [1 5 5 2 2 2]


### Tworzenie Numpy Array z list (Podobnie do przykładu)

In [7]:
samples = np.zeros((DATASET_SIZE,max_len))
for i in range(len(samples_int)):
    for j in range(len(samples_int[i])):
        samples[i,j] = np.array(samples_int[i][j])
print('SAMPLES\n\n',samples)
print(samples.shape)

SAMPLES

 [[0. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [2. 2. 0. ... 0. 0. 0.]
 ...
 [1. 5. 1. ... 0. 0. 0.]
 [1. 5. 1. ... 2. 0. 0.]
 [1. 5. 1. ... 0. 0. 0.]]
(200, 9)


In [8]:
samples = np.expand_dims(samples,axis=2) #add the third dimension
labels = np.array(labels,dtype=float)

print("Sample (for 123):\n",samples[123])
print("Sample decoded",to_text(samples[123]))
print("Label (output):",labels[123])

print('samples shape',samples.shape)
print('labels shape',labels.shape)

Sample (for 123):
 [[1.]
 [5.]
 [5.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]]
Sample decoded CXXIII   
Label (output): 123.0
samples shape (200, 9, 1)
labels shape (200,)


### Podział na zbiór testowy i treningowy

In [9]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE, random_state=1)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


### Funkcja check modelimport random

def check_model(verbose=0,how_many=5):
    pred = model.predict(samples)
    print('text => [predicted value] error=[error]')
    error = []
    for i in range(len(pred)):
        res = samples[i]
        error.append(abs(i-pred[i]))
        if verbose==1:
            train = ''
            if i in trainLabels: train='[T]'
            print(i,to_text(res),'=> {:.2f} error = {:.2f}'.format(pred[i,0],abs(i-pred[i,0])),train)
    if verbose<1: # if not verbose just display 'how_many' random samples
        for i in range(how_many):        
            x = random.randrange(DATASET_SIZE)
            res = samples[x]
            print(to_text(res),'=>  {:.2f} error = {:.2f}'.format(pred[x,0],abs(x-pred[x,0])))      
    print('Mean error =',np.mean(error))        
    return np.mean(error)
check_model(1)

In [10]:
import random

def check_model(verbose=0,how_many=5):
    pred = model.predict(samples)
    print('text => [predicted value] error=[error]')
    error = []
    for i in range(len(pred)):
        res = samples[i]
        error.append(abs(i-pred[i]))
        if verbose==1:
            train = ''
            if i in trainLabels: train='[T]'
            print(i,to_text(res),'=> {:.2f} error = {:.2f}'.format(pred[i,0],abs(i-pred[i,0])),train)
    if verbose<1: # if not verbose just display 'how_many' random samples
        for i in range(how_many):        
            x = random.randrange(DATASET_SIZE)
            res = samples[x]
            print(to_text(res),'=>  {:.2f} error = {:.2f}'.format(pred[x,0],abs(x-pred[x,0])))      
    print('Mean error =',np.mean(error))        
    return np.mean(error)

### Trenowanie sieci

In [11]:
EPOCHS=10
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(400):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    if x%20 == 0:
        print("\n{}/20 Epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
              format(x//20, num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
        check_model()
print("Done")

Training with 100 samples 10 epochs and batch_size= 25

0.0/20 Epochs: 10 - loss=8867.454, loss improvement=3279.935
text => [predicted value] error=[error]
XIX       =>  19.94 error = 0.94
XII       =>  19.94 error = 7.94
CLXXXVI   =>  19.94 error = 166.06
CXXII     =>  19.94 error = 102.06
CXX       =>  19.94 error = 100.06
Mean error = 81.649704

1.0/20 Epochs: 210 - loss=3124.075, loss improvement=29.884
text => [predicted value] error=[error]
LV        =>  85.75 error = 30.75
LXXXVIII  =>  85.75 error = 2.25
C         =>  85.71 error = 14.29
LXXVIII   =>  85.75 error = 7.75
XXVII     =>  85.75 error = 58.75
Mean error = 50.548355

2.0/20 Epochs: 410 - loss=3006.794, loss improvement= 0.117
text => [predicted value] error=[error]
CXXIII    =>  96.17 error = 26.83
CXV       =>  96.17 error = 18.83
LIX       =>  96.17 error = 37.17
CXV       =>  96.17 error = 18.83
XXX       =>  96.17 error = 66.17
Mean error = 49.574223

3.0/20 Epochs: 610 - loss=3006.942, loss improvement=-0.167
te

In [12]:
check_model(1)

text => [predicted value] error=[error]
0           => 0.01 error = 0.01 [T]
1 I         => 0.95 error = 0.05 [T]
2 II        => 1.97 error = 0.03 [T]
3 III       => 2.96 error = 0.04 [T]
4 IV        => 30.14 error = 26.14 
5 V         => 11.75 error = 6.75 
6 VI        => 6.20 error = 0.20 [T]
7 VII       => 6.48 error = 0.52 [T]
8 VIII      => 8.02 error = 0.02 [T]
9 IX        => 9.08 error = 0.08 [T]
10 X         => 9.96 error = 0.04 [T]
11 XI        => 34.38 error = 23.38 
12 XII       => 79.41 error = 67.41 
13 XIII      => 86.66 error = 73.66 
14 XIV       => 89.39 error = 75.39 
15 XV        => 15.90 error = 0.90 [T]
16 XVI       => 24.88 error = 8.88 
17 XVII      => 26.48 error = 9.48 
18 XVIII     => 26.31 error = 8.31 
19 XIX       => 84.66 error = 65.66 
20 XX        => 18.40 error = 1.60 [T]
21 XXI       => 20.67 error = 0.33 [T]
22 XXII      => 21.75 error = 0.25 [T]
23 XXIII     => 22.78 error = 0.22 [T]
24 XXIV      => 23.85 error = 0.15 [T]
25 XXV       => 24.93 error 

5.4787

### Wnioski
Klasyfikacja jest stosunkowo dokładna. Największe problemy pojawiają się dla małych liczb spoza zbioru treningowego, jednak sieć radzi sobie dobrze z większymi liczbami z zbioru testowego. Zjawisko przeuczenia nie jest aż tak bardzo widoczne. 